# Empirical corrections and Data cleaning for SFM 

### Libraries

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [2]:
import os
from pathlib import Path

path_cwd=Path.cwd()
path_input=str(path_cwd)+'/Data_input/'

import sys

sys.path.append(str(path_cwd)+'/Functions')
import baseline
import reading


### Dictionary

In [3]:
#this is parsing the inputfiles.csv file and creating a list of tuples with the following structure: [(name,tree,date,{Dict})]
from csv import DictReader, reader

with open(path_input+'inputfiles_2022.csv', encoding='utf-8-sig') as read_obj:
    dict_reader = DictReader(read_obj)
    list_of_dict = list(dict_reader)
 
    name_=[[list_of_dict[i].pop(key) for i in range(len(list_of_dict))]for key in ["Filename","Tree","Start_date","Soil"]] #extract string columns: Filename,Tree, Date (if applicable) .pop(key):removes specified element from list or dict 

    res = [dict([key, float(value)] for key, value in dicts.items()) for dicts in list_of_dict]#this will make all values integers ONLY WORKS if all values in file are numerical
    
    inputfiles=[(name_[0][i],name_[1][i],name_[2][i],name_[3][i],res[i]) for i in range(len(list_of_dict))] #list of tuples: [(name,tree,date,{Dict})]

### Main 
- Reading, cleaning and defining values for correction 
- Applying corrections from manual 
- Correction of baseline through linear regression
- Saving output files 

In [4]:
#General variables

#READING
SF_cols=['Date','Time','UO','UI','CO','CI','SFO','SFI','MaxTdO','MaxTuO','RiseTdO',
         'RiseTuO','RatioOut','MaxTdI','MaxTuI','RiseTdI','RiseTuI','RatioIn',
         'Pulse','BatteryVolt','BatteryT','ExternalPowerPresent','ExternalPowerVoltage',
         'ExternalPowerCurrent','Message']

#THERMAL DIFFUSIVITY
RHO=997 #water density (kg/m3)
CW=1200 #specific heat capacity of wood matrix (J kg-1 C-1)
CS=4182 #specific heat capacity of sap ( J kg-1 C-1 )
KS=0.5984 #thermal conductivity of water (J m-1 s-1 °C-1)

#CROSS SECTIONAL AREA
import math

pi=math.pi #define pi to use it 
BDI= 0 #bark depth at installation, if some needle is sticking out I can change value (cm) 

# WOUNDING COEFFICIENT
#here we assume a wound of 2 mm check manual if user want to change values
B=1.9216
b=1.8558
c=-0.0018
d=0.0003

In [5]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from distutils.version import LooseVersion
from packaging import version



In [6]:
name_sensor=[]
sf_out_total=[]

for i,(name,tree,ini_date,soil,options) in enumerate(inputfiles):
        from reading import read_sf1 
        SF=read_sf1(path_input,SF_cols,name)      

        ########THERMAL DIFFUSIVITY 
        BD,SD,DOB=options['Bark_depth'],options['SW_depth'],options['Diameter'] #bark depth (cm), SW depth (cm), diameter outer bark (cm)
        FW,DW,V=options['FW'],options['DW'],options['Vol'] #fresh weight (kg), #dry weight (kg), #volume (cm3)
        WC=FW-DW #WATER CONTENT (kg)
        MC=WC/DW #MOISTURE CONTENT (%)
        WD=(DW)/(V/1000000) #WOOD DENSITY (kg/m3)
        C= ((DW*CW)+CS*(FW-DW))/FW #SPECIFIC HEAT CAPACITY OF GREEN WOOD (J kg-1 C-1)
        FV= 1-(((WD*0.6536)+MC)/1000) #VOID FRACTION (no dimensions)
        a1=20*FV
        a2=21-a1
        KW=0.04182*a2 #THERMAL CONDUCTIVITY OF DRY WOOD (J m-1 s-1 C-1)
        a3=WD/RHO
        a4=MC*a3
        KGW=(KS*a4)+(KW*(1-a4)) #THERMAL CONDUCTIVITY green/fresh wood (J m-1 s-1 C-1)
        k=(KGW/(C*WD))*10000 #****THERMAL DIFFUSIVITY cm2/s****   

        ########SOIL AND ENVIRONMENTAL
        from reading import sf_env
        SF_W=sf_env(path_input,ini_date,SF,k)
        SF_W['HPVI_o'],SF_W['HPVO_o']=SF_W['HPVI'],SF_W['HPVO']


        #########48 correction
        if options['Output'] == 2:
                SF_1= SF_W.copy()
                hpvo=SF_1['HPVO'].loc['2022-05-25  22:30:00':'2022-06-29 14:00:00']*-1 
                hpvi=SF_1['HPVO'].loc['2022-05-25  22:30:00':'2022-06-29 14:00:00']*-1     
                SF_W['HPVO'].loc['2022-05-25  22:30:00':'2022-06-29 14:00:00']=hpvo
                SF_W['HPVI'].loc['2022-05-25  22:30:00':'2022-06-29 14:00:00']=hpvi 

        ########BASELINE CORRECTION 
        from baseline import baseline_LR_DLR 
        hpvi_lr,hpvi_dlr,TVC=baseline_LR_DLR(SF_W,nam='HPVI')
        hpvo_lr,hpvo_dlr,TVC=baseline_LR_DLR(SF_W,nam='HPVO')
        SF_corr=SF_W.drop(columns=['UO', 'UI', 'MaxTdO', 'MaxTuO', 'RiseTdO', 'RiseTuO', 'RatioOut','MaxTdI', 'MaxTuI', 'RiseTdI', 'RiseTuI', 'RatioIn'])
        SF_corr['HPVI_LR'],SF_corr['HPVI'],SF_corr['HPVO_LR'],SF_corr['HPVO'],SF_corr['TVC']=hpvi_lr,hpvi_dlr,hpvo_lr,hpvo_dlr,TVC #FOR THE REST OF THE ANALYSIS THE HPV DLR VALUES ARE USED

        ########WOUNDING COEFFICIENT CORRECTIONS 
        #DLR
        UO=SF_corr['HPVO'] #uncorrected heat pulse velocity for outer sensor (cm/h)
        UI=SF_corr['HPVI'] #uncorrected hpv for inner sensor (cm/h)
        VCO=b*UO+c*(UO.pow(2))+d*(UO.pow(3)) #eq 6 pp 25 in mmanual ()
        VCI=b*UI+c*(UI.pow(2))+d*(UI.pow(3))
        #LR
        UOLR=SF_corr['HPVO_LR'] #uncorrected heat pulse velocity for outer sensor (cm/h)
        UILR=SF_corr['HPVI_LR'] #uncorrected hpv for inner sensor (cm/h)
        VCOLR=b*UOLR+c*(UOLR.pow(2))+d*(UOLR.pow(3)) #eq 6 pp 25 in mmanual ()
        VCILR=b*UILR+c*(UILR.pow(2))+d*(UILR.pow(3))
        #orig
        UO_o=SF_corr['HPVO_o'] #uncorrected heat pulse velocity for outer sensor (cm/h)
        UI_o=SF_corr['HPVI_o'] #uncorrected hpv for inner sensor (cm/h)
        VCO_o=b*UO_o+c*(UO_o.pow(2))+d*(UO_o.pow(3)) #eq 6 pp 25 in mmanual ()
        VCI_o=b*UI_o+c*(UI_o.pow(2))+d*(UI_o.pow(3))


        ########SAP VELOCITY
        #DLR
        SF_corr['Sap velocity corr OUT (cm/h)']=(VCO*WD*(CW+MC*CS))/(RHO*CS)
        SF_corr['Sap velocity corr IN (cm/h)']=(VCI*WD*(CW+MC*CS))/(RHO*CS)
        #LR
        SF_corr['Sap velocity corr OUT_LR (cm/h)']=(VCOLR*WD*(CW+MC*CS))/(RHO*CS)
        SF_corr['Sap velocity corr IN_LR (cm/h)']=(VCILR*WD*(CW+MC*CS))/(RHO*CS)
        #orginal
        SF_corr['Sap velocity corr OUT_o (cm/h)']=(VCO_o*WD*(CW+MC*CS))/(RHO*CS)
        SF_corr['Sap velocity corr IN_o (cm/h)']=(VCI_o*WD*(CW+MC*CS))/(RHO*CS)


        ########CROSS SECTIONAL AREA
        BD,SD,DOB=options['Bark_depth'],options['SW_depth'],options['Diameter'] #bark depth (cm), SW depth (cm), diameter outer bark (cm)
        R=DOB/2 # tree radius (cm)
        XR=R-BD # xylem radius (sapwood+heartwood) (cm)
        HR=XR-SD # heartwood radius (cm)
        OSD=1.25-BDI # outer sensor depth (cm)
        ISD= 2.75-BDI # inner sensor depth (cm)
        MPD=(ISD-OSD)/2 #mid distance between 2 sensors (cm) =0.75

        AWo=OSD+MPD #outer annulus width (cm)
        AWi=ISD+MPD-AWo #inner annulus width (cm)
        MPR=XR-AWo # mid point radius 
        TSA=pi*((XR**2)-(HR**2)) #Total sap wood area (cm2)
        #print (TSA,tree)
        
        #A1:outer annulus area (cm2)
        #A2:inner annulus area (cm2)
        #A3:remainder annulus area (cm2)
        if (AWo>SD) & ((ISD+MPD)>SD): #just outer sensor, small SD
                A1=TSA
                A2=0
                A3=0
                SF_corr['Sap velocity (cm/h)']=SF_corr['Sap velocity corr OUT (cm/h)']
                SF_corr['Sap velocity_LR (cm/h)']=SF_corr['Sap velocity corr OUT_LR (cm/h)']
                SF_corr['Sap velocity_o (cm/h)']=SF_corr['Sap velocity corr OUT_o (cm/h)']
                
        elif (AWo<SD) & ((ISD+MPD)<SD): #in out and remainder area
                A1=pi*((XR**2)-(MPR**2))
                A2=pi*((MPR**2)-((MPR-AWi)**2))
                A3=pi*(((MPR-AWi)**2)-(HR**2))
                SF_corr['Sap velocity (cm/h)']=(SF_corr['Sap velocity corr IN (cm/h)']+SF_corr['Sap velocity corr OUT (cm/h)'])/2
                SF_corr['Sap velocity_LR (cm/h)']=(SF_corr['Sap velocity corr IN_LR (cm/h)']+SF_corr['Sap velocity corr OUT_LR (cm/h)'])/2      
                SF_corr['Sap velocity_o (cm/h)']=(SF_corr['Sap velocity corr IN_o (cm/h)']+SF_corr['Sap velocity corr OUT_o (cm/h)'])/2   

        else: # just outer but some extra area
                A1=pi*((XR**2)-(MPR**2))
                A2=0
                A3=pi*((MPR**2)-((HR)**2))
                SF_corr['Sap velocity (cm/h)']=SF_corr['Sap velocity corr OUT (cm/h)']
                SF_corr['Sap velocity_LR (cm/h)']=SF_corr['Sap velocity corr OUT_LR (cm/h)']
                SF_corr['Sap velocity_o (cm/h)']=SF_corr['Sap velocity corr OUT_o (cm/h)']


        ########SAPFLOW 
        #DLR
        SF_corr['Sap flow corr OUT (cm3/h)']=SF_corr['Sap velocity corr OUT (cm/h)']*A1 # (cm3/h)
        SF_corr['Sap flow corr IN (cm3/h)']=SF_corr['Sap velocity corr IN (cm/h)']*A2 #if we take density of water 0.001kg/cm3 we can multiply this value by 1/1000
        n=2 #n=1 if we assume remainder has same velocity as inner otherwise we assume linear decay (n=2)
        SF_corr['Sap flow corr REM (cm3/h)']=((SF_corr['Sap velocity corr IN (cm/h)']/n)*A3)
        SF_corr['Total SF (cm3/h)']=SF_corr['Sap flow corr OUT (cm3/h)']+SF_corr['Sap flow corr IN (cm3/h)']+SF_corr['Sap flow corr REM (cm3/h)']
        #LR
        SF_corr['Sap flow corr OUT_LR (cm3/h)']=SF_corr['Sap velocity corr OUT_LR (cm/h)']*A1 # (cm3/h)
        SF_corr['Sap flow corr IN_LR (cm3/h)']=SF_corr['Sap velocity corr IN_LR (cm/h)']*A2 #if we take density of water 0.001kg/cm3 we can multiply this value by 1/1000
        n=2 #n=1 if we assume remainder has same velocity as inner otherwise we assume linear decay (n=2)
        SF_corr['Sap flow corr REM_LR (cm3/h)']=((SF_corr['Sap velocity corr IN_LR (cm/h)']/n)*A3)
        SF_corr['Total SF_LR (cm3/h)']=SF_corr['Sap flow corr OUT_LR (cm3/h)']+SF_corr['Sap flow corr IN_LR (cm3/h)']+SF_corr['Sap flow corr REM_LR (cm3/h)']
        #original 
        SF_corr['Sap flow corr OUT_o (cm3/h)']=SF_corr['Sap velocity corr OUT_o (cm/h)']*A1 # (cm3/h)
        SF_corr['Sap flow corr IN_o (cm3/h)']=SF_corr['Sap velocity corr IN_o (cm/h)']*A2 #if we take density of water 0.001kg/cm3 we can multiply this value by 1/1000
        n=2 #n=1 if we assume remainder has same velocity as inner otherwise we assume linear decay (n=2)
        SF_corr['Sap flow corr REM_o (cm3/h)']=((SF_corr['Sap velocity corr IN_o (cm/h)']/n)*A3)
        SF_corr['Total SF_o (cm3/h)']=SF_corr['Sap flow corr OUT_o (cm3/h)']+SF_corr['Sap flow corr IN_o (cm3/h)']+SF_corr['Sap flow corr REM_o (cm3/h)']



        #######INTERPOLATION
        SF_corr['Total SF (cm3/h)']=SF_corr['Total SF (cm3/h)'].interpolate(option='polynomial',limit=10) # 
        SF_corr['Total SF_LR (cm3/h)']=SF_corr['Total SF_LR (cm3/h)'].interpolate(option='polynomial',limit=10)
        SF_corr['Total SF_o (cm3/h)']=SF_corr['Total SF_o (cm3/h)'].interpolate(option='polynomial',limit=10)

        SF_corr['Sap velocity (cm/h)']=SF_corr['Sap velocity (cm/h)'].interpolate(option='polynomial',limit=10)
        SF_corr['Sap velocity_LR (cm/h)']=SF_corr['Sap velocity_LR (cm/h)'].interpolate(option='polynomial',limit=10)
        SF_corr['Sap velocity_o (cm/h)']=SF_corr['Sap velocity_o (cm/h)'].interpolate(option='polynomial',limit=10)

        #########APPEND
        sf_out_w=SF_corr.copy().drop(columns=['HPVI', 'HPVO', 'HPVI_LR', 'HPVO_LR','HPVI_o', 'HPVO_o', 'TVC', 'Sap velocity corr OUT (cm/h)','Sap velocity corr IN (cm/h)','Sap velocity corr OUT_LR (cm/h)','Sap velocity corr IN_LR (cm/h)','Sap velocity corr OUT_o (cm/h)','Sap velocity corr IN_o (cm/h)','Sap flow corr OUT (cm3/h)','Sap flow corr IN (cm3/h)','Sap flow corr REM (cm3/h)','Sap flow corr OUT_LR (cm3/h)','Sap flow corr IN_LR (cm3/h)','Sap flow corr REM_LR (cm3/h)','Sap flow corr OUT_o (cm3/h)','Sap flow corr IN_o (cm3/h)','Sap flow corr REM_o (cm3/h)','Sap velocity_LR (cm/h)','Sap velocity_o (cm/h)','Total SF_LR (cm3/h)','Total SF_o (cm3/h)'])
        nam=os.path.splitext(name)[0]+ "-" + tree
        name_sensor.append(nam)
        sf_out_total.append(sf_out_w)

        #########OUTPUT FILES 
        path_cwd = str(Path.cwd())
        path_corrsf = path_cwd + "/Data_output/Incomplete_data/"
        path_TS= str(Path.cwd().parents[2]) #moves two directories up
        path_analysis= path_TS + "/ANALYSIS/Input_files/" 

        for dir in [path_corrsf,path_analysis]:
                sf_out_w.to_csv(dir + "SF_W_"+ nam + "-2022-incomplete" + ".csv", index = True)

        print("SF_W_"+ nam + "-2022-incomplete" + ".csv")

        #########PLOTTING
        # import warnings
        # warnings.filterwarnings("ignore", category=DeprecationWarning) 
        # print(tree)
        # fig = px.line()
        # fig.add_scatter(x=SF_corr.index.to_series(), y=SF_corr['Total SF (cm3/h)'] ,name='Total sap (cm3/h)')
        # fig.add_scatter(x=SF_corr.index.to_series(), y=SF_corr['Total SF_LR (cm3/h)'] ,name='Total sap_LR (cm3/h)')
        # fig.add_scatter(x=SF_corr.index.to_series(), y=SF_corr['Total SF_o (cm3/h)'] ,name='Total sap_original (cm3/h)')
        # fig.show()


        


SF_W_SFM1J20P-DF49GT-2022-incomplete.csv
SF_W_SFM1I60R-ES48GT-2022-incomplete.csv
SF_W_SFM1I60Q-DF21LS-2022-incomplete.csv
SF_W_SFM1K308-ES50LS-2022-incomplete.csv
SF_W_SFM1I60T-DF03US-2022-incomplete.csv
SF_W_SFM1J406-ES01US-2022-incomplete.csv
SF_W_SFM1J20O-DF27US-2022-incomplete.csv
SF_W_SFM1J20N-ES42US-2022-incomplete.csv


In [7]:
common_dates = set(sf_out_total[0].index)
#print(common_dates)

In [8]:
complete_sections = []
for df in sf_out_total:
    complete_mask = ~df['Total SF (cm3/h)'].isnull() & ~df['Sap velocity (cm/h)'].isnull()
    complete_sections.append(complete_mask)

In [9]:
best_regressions = []
for i, df in enumerate(sf_out_total):
    complete_mask = complete_sections[i]
    #x = df.values[complete_mask]
    best_r_squared = 0
    best_regression = None

    for j, other_df in enumerate(sf_out_total):
            if i != j:
                other_complete_mask = complete_sections[j]
                #other_x = other_df.values[other_complete_mask]


                #print (len(x), len(other_x))
                #x, y = x[complete_mask], x[complete_mask]


In [10]:


for name,df in zip(name_sensor,sf_out_total): 
    incomplete_df=[]
    incomplete_mask = df['Total SF (cm3/h)'].isnull()
    df['Total SF (cm3/h)'][~incomplete_mask]


